# State Farm

In [1]:
import sys
print(sys.version)
sys.version_info

2.7.12 |Anaconda 4.2.0 (64-bit)| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)

https://www.kaggle.com/c/state-farm-distracted-driver-detection


```
utils/
    vgg16.py
    utils.py
lesson1/
    redux.ipynb
    data/
        redux/
            train/
                cat.437.jpg
                dog.9924.jpg
                cat.1029.jpg
                dog.4374.jpg
            test/
                231.jpg
                325.jpg
                1235.jpg
                9923.jpg
```

In [3]:
#Verify we are in the lesson1 directory
# Should be .../deeplearning1/nbs
%pwd

u'/home/ubuntu/courses/deeplearning1/nbs'

In [4]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/statefarm/'
DATA_HOME_DIR

'/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/'

In [5]:
#Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '..'))
print(sys.path)
#import modules
from utils import *
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

['', '..', '/home/ubuntu/anaconda2/lib/python27.zip', '/home/ubuntu/anaconda2/lib/python2.7', '/home/ubuntu/anaconda2/lib/python2.7/plat-linux2', '/home/ubuntu/anaconda2/lib/python2.7/lib-tk', '/home/ubuntu/anaconda2/lib/python2.7/lib-old', '/home/ubuntu/anaconda2/lib/python2.7/lib-dynload', '/home/ubuntu/anaconda2/lib/python2.7/site-packages', '/home/ubuntu/anaconda2/lib/python2.7/site-packages/Sphinx-1.4.6-py2.7.egg', '/home/ubuntu/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/home/ubuntu/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/home/ubuntu/.ipython']


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [6]:
!ls | grep "\.py"

resnet50.py
utils.py
utils.pyc
vgg16bn.py
vgg16bn.pyc
vgg16.py
vgg16.pyc


## Action Plan
1. Create Validation and Sample sets
2. Rearrange image files into their respective directories 
3. Finetune and Train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

## Create validation set and sample

In [7]:
#Create directories
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p test/unknown

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm
mkdir: cannot create directory ‘valid’: File exists
mkdir: cannot create directory ‘results’: File exists


In [9]:
print(DATA_HOME_DIR)
%cd $DATA_HOME_DIR/train


/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/train


# Move from training set to validation set - only run these moving file blocks once

In [30]:
# Move about 1/10 to validation
# There are about 2000 pictures per category so move 200 per category to the validation folders
VAL_RATIO = .1
for subdir, dirs, files in os.walk(os.path.join(DATA_HOME_DIR, 'train/')):
    for dirr in sorted(dirs):
        # print(subdir, dirr)
        fulldir = os.path.join(subdir, dirr)
        %cd $fulldir
        TOTFILES = len([name for name in os.listdir('.') if os.path.isfile(name)])
        print "Total number of files in this subdir: {}".format(TOTFILES)
        g = glob('*.jpg')
        shuf = np.random.permutation(g)
        print(shuf[:5])
        DEST_VALI = os.path.join(DATA_HOME_DIR, 'valid', dirr)
        try: 
            os.makedirs(DEST_VALI)
        except:
            pass
        for i in range(int(VAL_RATIO*TOTFILES)): 
            if i < 2:
                print os.path.join(DEST_VALI, shuf[i])
            os.rename(shuf[i], os.path.join(DEST_VALI, shuf[i]))
            

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/train/c0
Total number of files in this subdir: 2489
['img_23182.jpg' 'img_29245.jpg' 'img_46818.jpg' 'img_75564.jpg' 'img_22984.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c0/img_23182.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c0/img_29245.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/train/c1
Total number of files in this subdir: 2267
['img_1242.jpg' 'img_49674.jpg' 'img_16371.jpg' 'img_6609.jpg' 'img_28270.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c1/img_1242.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c1/img_49674.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/train/c2
Total number of files in this subdir: 2317
['img_5784.jpg' 'img_92707.jpg' 'img_95514.jpg' 'img_41946.jpg' 'img_21964.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c2/img_5784.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/va

In [31]:
from shutil import copyfile

In [32]:
# Copy sample data to build network architecture off of - save a lot of time

for subdir, dirs, files in os.walk(os.path.join(DATA_HOME_DIR, 'train/')):
    for dirr in sorted(dirs):
        # print(subdir, dirr)
        fulldir = os.path.join(subdir, dirr)
        %cd $fulldir
        g = glob('*.jpg')
        shuf = np.random.permutation(g)
        print(shuf[:5])
        DEST_VALI = os.path.join(DATA_HOME_DIR, 'sample', 'train', dirr)
        try: 
            os.makedirs(DEST_VALI)
        except:
            pass
        for i in range(200): 
            if i < 2:
                print os.path.join(DEST_VALI, shuf[i])
            copyfile(shuf[i], os.path.join(DEST_VALI, shuf[i]))
            

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/train/c0
['img_13585.jpg' 'img_38088.jpg' 'img_13073.jpg' 'img_47700.jpg' 'img_72977.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c0/img_13585.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c0/img_38088.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c0/img_13073.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c0/img_47700.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c0/img_72977.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/train/c1
['img_4762.jpg' 'img_29506.jpg' 'img_55977.jpg' 'img_49415.jpg' 'img_64954.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c1/img_4762.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c1/img_29506.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/train/c1/img_55977.jpg
/home/ubuntu/courses/deeplearning1/nbs/d

In [33]:
%cd $DATA_HOME_DIR/valid

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid


In [35]:
# Copy sample data to build network architecture off of - save a lot of time

for subdir, dirs, files in os.walk(os.path.join(DATA_HOME_DIR, 'valid/')):
    for dirr in sorted(dirs):
        # print(subdir, dirr)
        fulldir = os.path.join(subdir, dirr)
        %cd $fulldir
        g = glob('*.jpg')
        shuf = np.random.permutation(g)
        print(shuf[:5])
        DEST_VALI = os.path.join(DATA_HOME_DIR, 'sample', 'valid', dirr)
        try: 
            os.makedirs(DEST_VALI)
        except:
            pass
        for i in range(50): 
            if i < 2:
                print os.path.join(DEST_VALI, shuf[i])
            copyfile(shuf[i], os.path.join(DEST_VALI, shuf[i]))

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c0
['img_86581.jpg' 'img_28578.jpg' 'img_91925.jpg' 'img_78518.jpg' 'img_12772.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/valid/c0/img_86581.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/valid/c0/img_28578.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c1
['img_29687.jpg' 'img_53660.jpg' 'img_66825.jpg' 'img_60268.jpg' 'img_33533.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/valid/c1/img_29687.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/valid/c1/img_53660.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/c2
['img_92876.jpg' 'img_96616.jpg' 'img_100646.jpg' 'img_42211.jpg' 'img_39809.jpg']
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/valid/c2/img_92876.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/sample/valid/c2/img_96616.jpg
/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/valid/

# Move test files into unknown dir

In [36]:
# Create single 'unknown' class for test set
%cd $DATA_HOME_DIR/test
%mv *.jpg unknown/

/home/ubuntu/courses/deeplearning1/nbs/data/statefarm/test


## Finetuning and Training

In [ ]:
%cd $DATA_HOME_DIR

#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

In [ ]:
#import Vgg16 helper class
vgg = Vgg16()

In [ ]:
#Set constants. You can experiment with no_of_epochs to improve the model
batch_size=64
no_of_epochs=3

In [ ]:
#Finetune the model
batches = vgg.get_batches(train_path, batch_size=batch_size)
val_batches = vgg.get_batches(valid_path, batch_size=batch_size*2)
vgg.finetune(batches)

#Not sure if we set this for all fits
# What the hell is this? - find out later
vgg.model.optimizer.lr = 0.01

In [ ]:
#Notice we are passing in the validation dataset to the fit() method
#For each epoch we test our model against the validation set
# ? Why do this three times? Just to see? There is no dependence etween the epoches in this method is there?
latest_weights_filename = None
for epoch in range(no_of_epochs):
    print "Running epoch: %d" % epoch
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch
    vgg.model.save_weights(results_path+latest_weights_filename)
print "Completed %s fit operations" % no_of_epochs

## Generate Predictions

Let's use our new model to make predictions on the test dataset

In [ ]:
batches, preds = vgg.test(test_path, batch_size = batch_size*2)

In [ ]:
#For every image, vgg.test() generates two probabilities 
#based on how we've ordered the cats/dogs directories.
#It looks like column one is cats and column two is dogs
print preds[:5]

filenames = batches.filenames
print filenames[:5]

In [ ]:
#You can verify the column ordering by viewing some images
from PIL import Image
Image.open(test_path + filenames[4])

In [ ]:
#Save our test results arrays so we can use them again later
# How does this save array work?
save_array(results_path + 'test_preds.dat', preds)
save_array(results_path + 'filenames.dat', filenames)


## Validate Predictions

Keras' *fit()* function conveniently shows us the value of the loss function, and the accuracy, after every epoch ("*epoch*" refers to one full run through all training examples). The most important metrics for us to look at are for the validation set, since we want to check for over-fitting. 

- **Tip**: with our first model we should try to overfit before we start worrying about how to reduce over-fitting - there's no point even thinking about regularization, data augmentation, etc if you're still under-fitting! (We'll be looking at these techniques shortly).

As well as looking at the overall metrics, it's also a good idea to look at examples of each of:
1. A few correct labels at random
2. A few incorrect labels at random
3. The most correct labels of each class (ie those with highest probability that are correct)
4. The most incorrect labels of each class (ie those with highest probability that are incorrect)
5. The most uncertain labels (ie those with probability closest to 0.5).

Let's see what we can learn from these examples. (In general, this is a particularly useful technique for debugging problems in the model. However, since this model is so simple, there may not be too much to learn at this stage.)

Calculate predictions on validation set, so we can find correct and incorrect examples:

In [ ]:
vgg.model.load_weights(results_path+latest_weights_filename)

In [ ]:
val_batches, probs = vgg.test(valid_path, batch_size = batch_size)

In [ ]:
filenames = val_batches.filenames
expected_labels = val_batches.classes #0 or 1

#Round our predictions to 0/1 to generate labels
our_predictions = probs[:,0]
our_labels = np.round(1-our_predictions)

In [ ]:
from keras.preprocessing import image

#Helper function to plot images by index in the validation set 
#Plots is a helper function in utils.py
def plots_idx(idx, titles=None):
    plots([image.load_img(valid_path + filenames[i]) for i in idx], titles=titles)
    
#Number of images to view for each visualization task
n_view = 4

In [ ]:
#1. A few correct labels at random
correct = np.where(our_labels==expected_labels)[0]
print "Found %d correct labels" % len(correct)
idx = permutation(correct)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
#2. A few incorrect labels at random
incorrect = np.where(our_labels!=expected_labels)[0]
print "Found %d incorrect labels" % len(incorrect)
idx = permutation(incorrect)[:n_view]
plots_idx(idx, our_predictions[idx])

In [ ]:
#3a. The images we most confident were cats, and are actually cats
correct_cats = np.where((our_labels==0) & (our_labels==expected_labels))[0]
print "Found %d confident correct cats labels" % len(correct_cats)
most_correct_cats = np.argsort(our_predictions[correct_cats])[::-1][:n_view]
plots_idx(correct_cats[most_correct_cats], our_predictions[correct_cats][most_correct_cats])

In [ ]:
#3b. The images we most confident were dogs, and are actually dogs
correct_dogs = np.where((our_labels==1) & (our_labels==expected_labels))[0]
print "Found %d confident correct dogs labels" % len(correct_dogs)
most_correct_dogs = np.argsort(our_predictions[correct_dogs])[:n_view]
plots_idx(correct_dogs[most_correct_dogs], our_predictions[correct_dogs][most_correct_dogs])

In [ ]:
#4a. The images we were most confident were cats, but are actually dogs
incorrect_cats = np.where((our_labels==0) & (our_labels!=expected_labels))[0]
print "Found %d incorrect cats" % len(incorrect_cats)
if len(incorrect_cats):
    most_incorrect_cats = np.argsort(our_predictions[incorrect_cats])[::-1][:n_view]
    plots_idx(incorrect_cats[most_incorrect_cats], our_predictions[incorrect_cats][most_incorrect_cats])

In [ ]:
#4b. The images we were most confident were dogs, but are actually cats
incorrect_dogs = np.where((our_labels==1) & (our_labels!=expected_labels))[0]
print "Found %d incorrect dogs" % len(incorrect_dogs)
if len(incorrect_dogs):
    most_incorrect_dogs = np.argsort(our_predictions[incorrect_dogs])[:n_view]
    plots_idx(incorrect_dogs[most_incorrect_dogs], our_predictions[incorrect_dogs][most_incorrect_dogs])

In [ ]:
#5. The most uncertain labels (ie those with probability closest to 0.5).
most_uncertain = np.argsort(np.abs(our_predictions-0.5))
plots_idx(most_uncertain[:n_view], our_predictions[most_uncertain])

Perhaps the most common way to analyze the result of a classification model is to use a [confusion matrix](http://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/). Scikit-learn has a convenient function we can use for this purpose:

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(expected_labels, our_labels)

We can just print out the confusion matrix, or we can show a graphical view (which is mainly useful for dependents with a larger number of categories).

In [ ]:
plot_confusion_matrix(cm, val_batches.class_indices)

## Submit Predictions to Kaggle!

Here's the format Kaggle requires for new submissions:
```
imageId,isDog
1242, .3984
3947, .1000
4539, .9082
2345, .0000
```

Kaggle wants the imageId followed by the probability of the image being a dog. Kaggle uses a metric called [Log Loss](http://wiki.fast.ai/index.php/Log_Loss) to evaluate your submission.

In [ ]:
#Load our test predictions from file
preds = load_array(results_path + 'test_preds.dat')
filenames = load_array(results_path + 'filenames.dat')

In [ ]:
#Grab the dog prediction column
isdog = preds[:,1]
print "Raw Predictions: " + str(isdog[:5])
print "Mid Predictions: " + str(isdog[(isdog < .6) & (isdog > .4)])
print "Edge Predictions: " + str(isdog[(isdog == 1) | (isdog == 0)])

[Log Loss](http://wiki.fast.ai/index.php/Log_Loss) doesn't support probability values of 0 or 1--they are undefined (and we have many). Fortunately, Kaggle helps us by offsetting our 0s and 1s by a very small value. So if we upload our submission now we will have lots of .99999999 and .000000001 values. This seems good, right?

Not so. There is an additional twist due to how log loss is calculated--log loss rewards predictions that are confident and correct (p=.9999,label=1), but it punishes predictions that are confident and wrong far more (p=.0001,label=1). See visualization below.

In [ ]:
#Visualize Log Loss when True value = 1
#y-axis is log loss, x-axis is probabilty that label = 1
#As you can see Log Loss increases rapidly as we approach 0
#But increases slowly as our predicted probability gets closer to 1
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import log_loss

x = [i*.0001 for i in range(1,10000)]
y = [log_loss([1],[[i*.0001,1-(i*.0001)]],eps=1e-15) for i in range(1,10000,1)]

plt.plot(x, y)
plt.axis([-.05, 1.1, -.8, 10])
plt.title("Log Loss when true label = 1")
plt.xlabel("predicted probability")
plt.ylabel("log loss")

plt.show()

In [ ]:
#So to play it safe, we use a sneaky trick to round down our edge predictions
#Swap all ones with .95 and all zeros with .05
isdog = isdog.clip(min=0.05, max=0.95)

In [ ]:
#Extract imageIds from the filenames in our test/unknown directory 
filenames = batches.filenames
ids = np.array([int(f[8:f.find('.')]) for f in filenames])

Here we join the two columns into an array of [imageId, isDog]

In [ ]:
subm = np.stack([ids,isdog], axis=1)
subm[:5]

In [ ]:
%cd $DATA_HOME_DIR
submission_file_name = 'submission1.csv'
np.savetxt(submission_file_name, subm, fmt='%d,%.5f', header='id,label', comments='')

In [ ]:
from IPython.display import FileLink
%cd $LESSON_HOME_DIR
FileLink(DATA_HOME_DIR+'/'+submission_file_name)

You can download this file and submit on the Kaggle website or use the Kaggle command line tool's "submit" method.